In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
# Set random seed
tf.random.set_seed(42)

In [4]:
# === Data Paths ===
train_dir = '/content/drive/MyDrive/train'
val_dir = '/content/drive/MyDrive/val'
test_dir = '/content/drive/MyDrive/val'
img_size = (224, 224)
batch_size = 32

In [5]:
# === Data Augmentation (same as original) ===
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)
val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

num_classes = len(train_gen.class_indices)

Found 697 images belonging to 9 classes.
Found 181 images belonging to 9 classes.
Found 181 images belonging to 9 classes.


In [6]:
# === Manual Class Weights (balanced) in TensorFlow ===
# Equivalent to sklearn.utils.class_weight.compute_class_weight('balanced', ...)
def compute_balanced_class_weights_tf(labels, num_classes: int):
    # labels is a NumPy array from Keras generator; convert to tf
    y = tf.convert_to_tensor(labels, dtype=tf.int32)
    counts = tf.math.bincount(y, minlength=num_classes, maxlength=num_classes)
    n_samples = tf.cast(tf.size(y), tf.float32)
    n_classes = tf.cast(num_classes, tf.float32)
    counts_f = tf.cast(counts, tf.float32)
    # avoided div-by-zero
    counts_safe = tf.where(counts_f > 0.0, counts_f, tf.ones_like(counts_f))
    weights = n_samples / (n_classes * counts_safe)
    weights_list = weights.numpy().tolist()  # convert to Python floats
    return {i: float(w) for i, w in enumerate(weights_list)}

labels = train_gen.classes  # integer labels
class_weights = compute_balanced_class_weights_tf(labels, num_classes)

In [7]:
# === Built Model (same as original) ===
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = True  # Unfreeze all layers

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
predictions = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
# === Callbacks ===
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2, min_lr=1e-6)

In [9]:
# === Train (same API) ===
history = model.fit(
    train_gen,
    epochs=30,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 726s 29s/step - accuracy: 0.1962 - loss: 2.6951 - val_accuracy: 0.3812 - val_loss: 1.8784 - learning_rate: 1.0000e-04
Epoch 2/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 620s 28s/step - accuracy: 0.6207 - loss: 1.1521 - val_accuracy: 0.4862 - val_loss: 1.4983 - learning_rate: 1.0000e-04
Epoch 3/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 629s 29s/step - accuracy: 0.7104 - loss: 0.7557 - val_accuracy: 0.5304 - val_loss: 1.3255 - learning_rate: 1.0000e-04
Epoch 4/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 627s 29s/step - accuracy: 0.8242 - loss: 0.5115 - val_accuracy: 0.6022 - val_loss: 1.1096 - learning_rate: 1.0000e-04
Epoch 5/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 620s 28s/step - accuracy: 0.8637 - loss: 0.3749 - val_accuracy: 0.6740 - val_loss: 0.8846 - learning_rate: 1.0000e-04
Epoch 6/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 625s 28s/step - accuracy: 0.8839 - loss: 0.3746 - val_accuracy: 0.7127 - val_loss: 0.8614 - learning_rate: 1.0000e-04
Epoch 7/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 618s 28s/step - accuracy: 

In [10]:
# === Evaluate (same API) ===
loss, acc = model.evaluate(test_gen)
print(f"Final Test Accuracy: {acc * 100:.2f}%")

181/181 ━━━━━━━━━━━━━━━━━━━━ 34s 189ms/step - accuracy: 0.7243 - loss: 0.8366
Final Test Accuracy: 81.77%


In [11]:
# === Predictions (TensorFlow ops, no numpy import) ===
# model.predict returns a NumPy array under the hood, but we operate with TF tensors after conversion.
Y_pred = model.predict(test_gen, steps=test_gen.samples, verbose=0)
Y_pred_tf = tf.convert_to_tensor(Y_pred)
y_pred = tf.math.argmax(Y_pred_tf, axis=1, output_type=tf.int32)

In [12]:
# True labels from generator (NumPy array), converted to tf
y_true = tf.convert_to_tensor(test_gen.classes, dtype=tf.int32)
class_names = list(test_gen.class_indices.keys())
num_classes = len(class_names)

In [13]:
# === Confusion Matrix (TensorFlow) ===
cm_tf = tf.math.confusion_matrix(y_true, y_pred, num_classes=num_classes, dtype=tf.int32)

# Pretty-print confusion matrix as text (no plotting libs)
def print_confusion_matrix(cm_tensor: tf.Tensor, class_names):
    cm = cm_tensor.numpy()  # to print cleanly
    k = cm.shape[0]
    width = max(5, max(len(c) for c in class_names) + 2)
    header = " " * width + "".join([f"{c:>{width}}" for c in class_names])
    print("\nConfusion Matrix (rows=true, cols=pred):")
    print(header)
    for i in range(k):
        row = f"{class_names[i]:>{width}}" + "".join([f"{int(cm[i, j]):>{width}d}" for j in range(k)])
        print(row)

print_confusion_matrix(cm_tf, class_names)


Confusion Matrix (rows=true, cols=pred):
                                       Actinic keratosis           Atopic Dermatitis            Benign keratosis              Dermatofibroma           Melanocytic nevus                    Melanoma     Squamous cell carcinoma  Tinea Ringworm Candidiasis             Vascular lesion
           Actinic keratosis                           7                           0                           0                           3                           0                           0                           9                           1                           0
           Atopic Dermatitis                           0                          20                           0                           0                           0                           0                           0                           1                           0
            Benign keratosis                           0                           0                          20   

In [14]:
# === Classification Report (TensorFlow) ===
def classification_report_tf(y_true_tf: tf.Tensor, y_pred_tf: tf.Tensor, class_names):
    num_classes = len(class_names)
    cm = tf.math.confusion_matrix(y_true_tf, y_pred_tf, num_classes=num_classes, dtype=tf.int32)
    tp = tf.linalg.diag_part(cm)
    pred_sum = tf.reduce_sum(cm, axis=0)  # predicted positives per class
    true_sum = tf.reduce_sum(cm, axis=1)  # actual positives per class

    tp_f = tf.cast(tp, tf.float32)
    pred_sum_f = tf.cast(pred_sum, tf.float32)
    true_sum_f = tf.cast(true_sum, tf.float32)

    precision = tf.where(pred_sum_f > 0, tp_f / pred_sum_f, tf.zeros_like(pred_sum_f))
    recall = tf.where(true_sum_f > 0, tp_f / true_sum_f, tf.zeros_like(true_sum_f))
    denom = precision + recall
    f1 = tf.where(denom > 0, 2.0 * precision * recall / denom, tf.zeros_like(denom))

    support = true_sum
    total = tf.reduce_sum(true_sum)
    accuracy = tf.cast(tf.reduce_sum(tp), tf.float32) / tf.cast(total, tf.float32)

    macro_p = tf.reduce_mean(precision)
    macro_r = tf.reduce_mean(recall)
    macro_f1 = tf.reduce_mean(f1)
    weights = tf.where(total > 0, tf.cast(support, tf.float32) / tf.cast(total, tf.float32), tf.zeros_like(tf.cast(support, tf.float32)))
    weighted_p = tf.reduce_sum(weights * precision)
    weighted_r = tf.reduce_sum(weights * recall)
    weighted_f1 = tf.reduce_sum(weights * f1)

    # Printed nicely
    header = f"{'Class':<20}{'Precision':>12}{'Recall':>12}{'F1-Score':>12}{'Support':>10}"
    print("\nClassification Report:")
    print(header)
    print("-" * len(header))
    prec_np = precision.numpy()
    rec_np = recall.numpy()
    f1_np = f1.numpy()
    sup_np = support.numpy()
    for i, name in enumerate(class_names):
        print(f"{name:<20}{prec_np[i]:>12.4f}{rec_np[i]:>12.4f}{f1_np[i]:>12.4f}{int(sup_np[i]):>10d}")
    print("")
    print(f"{'Accuracy':<20}{'':>12}{'':>12}{accuracy.numpy():>12.4f}{int(total.numpy()):>10d}")
    print(f"{'Macro avg':<20}{macro_p.numpy():>12.4f}{macro_r.numpy():>12.4f}{macro_f1.numpy():>12.4f}{int(total.numpy()):>10d}")
    print(f"{'Weighted avg':<20}{weighted_p.numpy():>12.4f}{weighted_r.numpy():>12.4f}{weighted_f1.numpy():>12.4f}{int(total.numpy()):>10d}")

classification_report_tf(y_true, y_pred, class_names)


Classification Report:
Class                  Precision      Recall    F1-Score   Support
------------------------------------------------------------------
Actinic keratosis         0.8750      0.3500      0.5000        20
Atopic Dermatitis         0.9524      0.9524      0.9524        21
Benign keratosis          0.9524      1.0000      0.9756        20
Dermatofibroma            0.7308      0.9500      0.8261        20
Melanocytic nevus         0.7727      0.8500      0.8095        20
Melanoma                  0.7500      0.4500      0.5625        20
Squamous cell carcinoma      0.5806      0.9000      0.7059        20
Tinea Ringworm Candidiasis      0.9048      0.9500      0.9268        20
Vascular lesion           1.0000      0.9500      0.9744        20

Accuracy                                          0.8177       181
Macro avg                 0.8354      0.8169      0.8037       181
Weighted avg              0.8361      0.8177      0.8045       181


In [15]:
# === Save Model ===
os.makedirs('/content/drive/MyDrive/final_project', exist_ok=True)
save_path = '/content/drive/MyDrive/final_project/DenseNet_No_Library_Possible.h5'
model.save(save_path)
print(f"\nModel saved to {save_path}")


Model saved to /content/drive/MyDrive/final_project/DenseNet_No_Library_Possible.h5
